In [1]:
import scanpy as sc
import pandas as pd
import pickle
import scnsbm
import time
import anndata
import sklearn.metrics
import statistics
%matplotlib notebook
%pylab

/home/morelli/anaconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex
/home/morelli/anaconda3/envs/scrna/lib/python3.7/site-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/home/morelli/anaconda3/envs/scrna/lib/python3.7/site-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/home/morelli/anaconda3/envs/scrna/lib/python3.7/site-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/home/morelli/anaconda3/envs/scrna/lib/python3.7/site-packages/graph_tool/all.py:40: RuntimeWarning: Error im

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


In [54]:
df = pd.read_table("matrix_mixology/sc_dropseq.count.csv.gz", compression="gzip", sep=",")
nomi=pd.read_csv('regressed_genes/nomi_geni.txt.gz', compression= 'gzip')
cell_cycle_genes = [x.strip() for x in open('regressed_genes/regev_lab_cell_cycle_genes.txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
ensg_s_genes =[nomi.loc[idx, "Gene stable ID"] for idx in nomi.index if nomi.loc[idx, "Gene name"] in s_genes]
ensg_g2m_genes =[nomi.loc[idx, "Gene stable ID"] for idx in nomi.index if nomi.loc[idx, "Gene name"] in g2m_genes]
adata = anndata.AnnData(df.T)#.values.T, obs=df.columns, var=df.index)
mito_genes = [nomi.loc[idx, "Gene stable ID"] for idx in nomi.index if nomi.loc[idx, "Gene name"].startswith("MT-") and nomi.loc[idx, "Gene stable ID"] in adata.var_names]
 
adata.obs['percent_mito'] = np.sum(adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1) 
adata.obs['n_counts'] = adata.X.sum(axis=1)
np.sum(adata[:, mito_genes].X, axis=1)                                            
                                            

adata = adata[adata.obs['percent_mito'] < 0.05, :]
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)
adata.raw = adata
sc.tl.score_genes_cell_cycle(adata, s_genes=ensg_s_genes, g2m_genes=ensg_g2m_genes)

sc.pp.highly_variable_genes(adata, min_mean=0.05, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var['highly_variable']]
sc.pp.regress_out(adata, ['S_score', 'G2M_score','n_counts', 'percent_mito'])
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
cell_annotation= pd.read_table('matrix_mixology/sc_dropseq.metadata.csv.gz', compression="gzip", sep=",", index_col=0)
adata.obs.loc[:, "cell_line"] = cell_annotation.loc[adata.obs_names, "cell_line"]
adata.write('mi_dropseq/adata_base.h5ad')

Trying to set attribute `.obs` of view, copying.


/home/morelli/anaconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1172: ImplicitModificationWarning: Initializing view as actual.
  "Initializing view as actual.", ImplicitModificationWarning
Trying to set attribute `.obs` of view, copying.
... storing 'phase' as categorical
... storing 'cell_line' as categorical


In [2]:
adata=sc.read('mi_dropseq/adata_base.h5ad')

In [7]:
for n_pcs in [5, 10, 20, 30]:
    n_neighbors = np.arange(5, 55, 5)
    MI = dict.fromkeys(n_neighbors)
    for n in n_neighbors:
        print(n_pcs, n)
        MI[n] = np.zeros((5, 10))
        sc.pp.neighbors(adata, n_pcs=n_pcs, n_neighbors=n)
        for x in range(5):
            scnsbm.inference.nested_model(adata, save_state=False, 
                              collect_marginals=False,
                              equilibrate=False)
            for l in range(1, 10):
                ln = f'nsbm_level_{l}'
                v = sklearn.metrics.adjusted_mutual_info_score(adata.obs['cell_line'], adata.obs[ln])
                MI[n][x, l] = v
    pickle.dump(MI, open(f'mi_dropseq/mi_vs_cell_line/MI_{n_pcs}pcs_dropseq.pkl', 'wb'), 2) 

5 5
5 10
5 15
5 20
5 25
5 30
5 35
5 40
5 45
5 50
10 5
10 10
10 15
10 20
10 25
10 30
10 35
10 40
10 45
10 50
20 5
20 10
20 15
20 20
20 25
20 30
20 35
20 40
20 45
20 50
30 5
30 10
30 15
30 20
30 25
30 30
30 35
30 40
30 45
30 50


In [56]:
nei=[x for x in range(5,51,5)]
adatone=[]
for x in nei:
    adata=sc.read('mi_dropseq/adata_base.h5ad')
    sc.pp.neighbors(adata, n_neighbors=x, n_pcs=40)
    adatone.append(adata)
adatone[0].write('mi_dropseq/mi_vs_cell_line/adata_5.h5ad')
adatone[1].write('mi_dropseq/mi_vs_cell_line/adata_10.h5ad')
adatone[2].write('mi_dropseq/mi_vs_cell_line/adata_15.h5ad')
adatone[3].write('mi_dropseq/mi_vs_cell_line/adata_20.h5ad')
adatone[4].write('mi_dropseq/mi_vs_cell_line/adata_25.h5ad')
adatone[5].write('mi_dropseq/mi_vs_cell_line/adata_30.h5ad')
adatone[6].write('mi_dropseq/mi_vs_cell_line/adata_35.h5ad')
adatone[7].write('mi_dropseq/mi_vs_cell_line/adata_40.h5ad')
adatone[8].write('mi_dropseq/mi_vs_cell_line/adata_45.h5ad')
adatone[9].write('mi_dropseq/mi_vs_cell_line/adata_50.h5ad')

In [59]:
adata5=sc.read('mi_dropseq/mi_vs_cell_line/adata_5.h5ad')
adata10=sc.read('mi_dropseq/mi_vs_cell_line/adata_10.h5ad')
adata15=sc.read('mi_dropseq/mi_vs_cell_line/adata_15.h5ad')
adata20=sc.read('mi_dropseq/mi_vs_cell_line/adata_20.h5ad')
adata25=sc.read('mi_dropseq/mi_vs_cell_line/adata_25.h5ad')
adata30=sc.read('mi_dropseq/mi_vs_cell_line/adata_30.h5ad')
adata35=sc.read('mi_dropseq/mi_vs_cell_line/adata_35.h5ad')
adata40=sc.read('mi_dropseq/mi_vs_cell_line/adata_40.h5ad')
adata45=sc.read('mi_dropseq/mi_vs_cell_line/adata_45.h5ad')
adata50=sc.read('mi_dropseq/mi_vs_cell_line/adata_50.h5ad')

# ADATA5

In [ ]:
scnsbm.inference.nested_model(adata5)
adata5.write('mi_dropseq/mi_vs_cell_line/adata_5a.h5ad')
scnsbm.inference.nested_model(adata5,collect_marginals=False)
adata5.write('mi_dropseq/mi_vs_cell_line/adata_5b.h5ad')
scnsbm.inference.nested_model(adata5,collect_marginals=False)
adata5.write('mi_dropseq/mi_vs_cell_line/adata_5c.h5ad')
scnsbm.inference.nested_model(adata5,collect_marginals=False)
adata5.write('mi_dropseq/mi_vs_cell_line/adata_5d.h5ad')
scnsbm.inference.nested_model(adata5,collect_marginals=False)
adata5.write('mi_dropseq/mi_vs_cell_line/adata_5e.h5ad')

In [ ]:
adata5a=sc.read('mi_dropseq/mi_vs_cell_line/adata_5a.h5ad')
adata5b=sc.read('mi_dropseq/mi_vs_cell_line/adata_5b.h5ad')
adata5c=sc.read('mi_dropseq/mi_vs_cell_line/adata_5c.h5ad')
adata5d=sc.read('mi_dropseq/mi_vs_cell_line/adata_5d.h5ad')
adata5e=sc.read('mi_dropseq/mi_vs_cell_line/adata_5e.h5ad')

### MI5

In [ ]:
nsbm_levels = [x for x in adata5a.obs_keys() if x.startswith("nsbm")]
foobars=['foobar5a','foobar5b','foobar5c','foobar5d','foobar5e']
foobar5a= [sklearn.metrics.adjusted_mutual_info_score(adata5a.obs.loc[:, NL], adata5a.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar5b= [sklearn.metrics.adjusted_mutual_info_score(adata5b.obs.loc[:, NL], adata5b.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar5c= [sklearn.metrics.adjusted_mutual_info_score(adata5c.obs.loc[:, NL], adata5c.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar5d= [sklearn.metrics.adjusted_mutual_info_score(adata5d.obs.loc[:, NL], adata5d.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar5e= [sklearn.metrics.adjusted_mutual_info_score(adata5e.obs.loc[:, NL], adata5e.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
MI_5=pd.DataFrame(None,columns=foobars,index=nsbm_levels)
MI_5.loc[:,'foobar5a']=foobar5a
MI_5.loc[:,'foobar5b']=foobar5b
MI_5.loc[:,'foobar5c']=foobar5c
MI_5.loc[:,'foobar5d']=foobar5d
MI_5.loc[:,'foobar5e']=foobar5e
mean_5=[mean(MI_5.iloc[x,:]) for x in range(len(nsbm_levels))]
sd_5=[statistics.stdev(MI_5.iloc[x,:]) for x in range(len(nsbm_levels))]
MI_5['mean_5']=mean_5
MI_5['standard_deviation_5']=sd_5
pickle.dump(MI_5, open('mi_dropseq/mi_vs_cell_line/mi_5.pkl','wb'),2)
print(MI_5)
MI_5.iloc[:,0:4].T.boxplot()

# ADATA10

In [ ]:
scnsbm.inference.nested_model(adata10)
adata10.write('mi_dropseq/mi_vs_cell_line/adata_10a.h5ad')
scnsbm.inference.nested_model(adata10,collect_marginals=False)
adata10.write('mi_dropseq/mi_vs_cell_line/adata_10b.h5ad')
scnsbm.inference.nested_model(adata10,collect_marginals=False)
adata10.write('mi_dropseq/mi_vs_cell_line/adata_10c.h5ad')
scnsbm.inference.nested_model(adata10,collect_marginals=False)
adata10.write('mi_dropseq/mi_vs_cell_line/adata_10d.h5ad')
scnsbm.inference.nested_model(adata10,collect_marginals=False)
adata10.write('mi_dropseq/mi_vs_cell_line/adata_10e.h5ad')

In [ ]:
adata10a=sc.read('mi_dropseq/mi_vs_cell_line/adata_10a.h5ad')
adata10b=sc.read('mi_dropseq/mi_vs_cell_line/adata_10b.h5ad')
adata10c=sc.read('mi_dropseq/mi_vs_cell_line/adata_10c.h5ad')
adata10d=sc.read('mi_dropseq/mi_vs_cell_line/adata_10d.h5ad')
adata10e=sc.read('mi_dropseq/mi_vs_cell_line/adata_10e.h5ad')

### MI10

In [ ]:
nsbm_levels = [x for x in adata10a.obs_keys() if x.startswith("nsbm")]
foobars=['foobar10a','foobar10b','foobar10c','foobar10d','foobar10e']
foobar10a= [sklearn.metrics.adjusted_mutual_info_score(adata10a.obs.loc[:, NL], adata10a.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar10b= [sklearn.metrics.adjusted_mutual_info_score(adata10b.obs.loc[:, NL], adata10b.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar10c= [sklearn.metrics.adjusted_mutual_info_score(adata10c.obs.loc[:, NL], adata10c.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar10d= [sklearn.metrics.adjusted_mutual_info_score(adata10d.obs.loc[:, NL], adata10d.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar10e= [sklearn.metrics.adjusted_mutual_info_score(adata10e.obs.loc[:, NL], adata10e.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
MI_10=pd.DataFrame(None,columns=foobars,index=nsbm_levels)
MI_10.loc[:,'foobar10a']=foobar10a
MI_10.loc[:,'foobar10b']=foobar10b
MI_10.loc[:,'foobar10c']=foobar10c
MI_10.loc[:,'foobar10d']=foobar10d
MI_10.loc[:,'foobar10e']=foobar10e
mean_10=[mean(MI_10.iloc[x,:]) for x in range(len(nsbm_levels))]
sd_10=[statistics.stdev(MI_10.iloc[x,:]) for x in range(len(nsbm_levels))]
MI_10['mean_10']=mean_10
MI_10['standard_deviation_10']=sd_10
pickle.dump(MI_10, open('mi_dropseq/mi_vs_cell_line/mi_10.pkl','wb'),2)
print(MI_10)
MI_10.iloc[:,0:4].T.boxplot()

# ADATA15

In [ ]:
scnsbm.inference.nested_model(adata15)
adata15.write('mi_dropseq/mi_vs_cell_line/adata_15a.h5ad')
scnsbm.inference.nested_model(adata15,collect_marginals=False)
adata15.write('mi_dropseq/mi_vs_cell_line/adata_15b.h5ad')
scnsbm.inference.nested_model(adata15,collect_marginals=False)
adata15.write('mi_dropseq/mi_vs_cell_line/adata_15c.h5ad')
scnsbm.inference.nested_model(adata15,collect_marginals=False)
adata15.write('mi_dropseq/mi_vs_cell_line/adata_15d.h5ad')
scnsbm.inference.nested_model(adata15,collect_marginals=False)
adata15.write('mi_dropseq/mi_vs_cell_line/adata_15e.h5ad')

In [ ]:
adata15a=sc.read('mi_dropseq/mi_vs_cell_line/adata_15a.h5ad')
adata15b=sc.read('mi_dropseq/mi_vs_cell_line/adata_15b.h5ad')
adata15c=sc.read('mi_dropseq/mi_vs_cell_line/adata_15c.h5ad')
adata15d=sc.read('mi_dropseq/mi_vs_cell_line/adata_15d.h5ad')
adata15e=sc.read('mi_dropseq/mi_vs_cell_line/adata_15e.h5ad')

### MI15

In [ ]:
nsbm_levels = [x for x in adata15a.obs_keys() if x.startswith("nsbm")]
foobars=['foobar15a','foobar15b','foobar15c','foobar15d','foobar15e']
foobar15a= [sklearn.metrics.adjusted_mutual_info_score(adata15a.obs.loc[:, NL], adata15a.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar15b= [sklearn.metrics.adjusted_mutual_info_score(adata15b.obs.loc[:, NL], adata15b.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar15c= [sklearn.metrics.adjusted_mutual_info_score(adata15c.obs.loc[:, NL], adata15c.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar15d= [sklearn.metrics.adjusted_mutual_info_score(adata15d.obs.loc[:, NL], adata15d.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar15e= [sklearn.metrics.adjusted_mutual_info_score(adata15e.obs.loc[:, NL], adata15e.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
MI_15=pd.DataFrame(None,columns=foobars,index=nsbm_levels)
MI_15.loc[:,'foobar15a']=foobar15a
MI_15.loc[:,'foobar15b']=foobar15b
MI_15.loc[:,'foobar15c']=foobar15c
MI_15.loc[:,'foobar15d']=foobar15d
MI_15.loc[:,'foobar15e']=foobar15e
mean_15=[mean(MI_15.iloc[x,:]) for x in range(len(nsbm_levels))]
sd_15=[statistics.stdev(MI_15.iloc[x,:]) for x in range(len(nsbm_levels))]
MI_15['mean_15']=mean_15
MI_15['standard_deviation_15']=sd_15
pickle.dump(MI_15, open('mi_dropseq/mi_vs_cell_line/mi_15.pkl','wb'),2)
print(MI_15)
MI_15.iloc[:,0:4].T.boxplot()

# ADATA20

In [ ]:
scnsbm.inference.nested_model(adata20)
adata20.write('mi_dropseq/mi_vs_cell_line/adata_20a.h5ad')
scnsbm.inference.nested_model(adata20,collect_marginals=False)
adata20.write('mi_dropseq/mi_vs_cell_line/adata_20b.h5ad')
scnsbm.inference.nested_model(adata20,collect_marginals=False)
adata20.write('mi_dropseq/mi_vs_cell_line/adata_20c.h5ad')
scnsbm.inference.nested_model(adata20,collect_marginals=False)
adata20.write('mi_dropseq/mi_vs_cell_line/adata_20d.h5ad')
scnsbm.inference.nested_model(adata20,collect_marginals=False)
adata20.write('mi_dropseq/mi_vs_cell_line/adata_20e.h5ad')

In [ ]:
adata20a=sc.read('mi_dropseq/mi_vs_cell_line/adata_20a.h5ad')
adata20b=sc.read('mi_dropseq/mi_vs_cell_line/adata_20b.h5ad')
adata20c=sc.read('mi_dropseq/mi_vs_cell_line/adata_20c.h5ad')
adata20d=sc.read('mi_dropseq/mi_vs_cell_line/adata_20d.h5ad')
adata20e=sc.read('mi_dropseq/mi_vs_cell_line/adata_20e.h5ad')

### MI20

In [ ]:
nsbm_levels = [x for x in adata20a.obs_keys() if x.startswith("nsbm")]
foobars=['foobar20a','foobar20b','foobar20c','foobar20d','foobar20e']
foobar20a= [sklearn.metrics.adjusted_mutual_info_score(adata20a.obs.loc[:, NL], adata20a.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar20b= [sklearn.metrics.adjusted_mutual_info_score(adata20b.obs.loc[:, NL], adata20b.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar20c= [sklearn.metrics.adjusted_mutual_info_score(adata20c.obs.loc[:, NL], adata20c.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar20d= [sklearn.metrics.adjusted_mutual_info_score(adata20d.obs.loc[:, NL], adata20d.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar20e= [sklearn.metrics.adjusted_mutual_info_score(adata20e.obs.loc[:, NL], adata20e.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
MI_20=pd.DataFrame(None,columns=foobars,index=nsbm_levels)
MI_20.loc[:,'foobar20a']=foobar20a
MI_20.loc[:,'foobar20b']=foobar20b
MI_20.loc[:,'foobar20c']=foobar20c
MI_20.loc[:,'foobar20d']=foobar20d
MI_20.loc[:,'foobar20e']=foobar20e
mean_20=[mean(MI_20.iloc[x,:]) for x in range(len(nsbm_levels))]
sd_20=[statistics.stdev(MI_20.iloc[x,:]) for x in range(len(nsbm_levels))]
MI_20['mean_20']=mean_20
MI_20['standard_deviation_20']=sd_20
pickle.dump(MI_20, open('mi_dropseq/mi_vs_cell_line/mi_20.pkl','wb'),2)
print(MI_20)
MI_20.iloc[:,0:4].T.boxplot()

# ADATA25

In [ ]:
scnsbm.inference.nested_model(adata25)
adata25.write('mi_dropseq/mi_vs_cell_line/adata_25a.h5ad')
scnsbm.inference.nested_model(adata25,collect_marginals=False)
adata25.write('mi_dropseq/mi_vs_cell_line/adata_25b.h5ad')
scnsbm.inference.nested_model(adata25,collect_marginals=False)
adata25.write('mi_dropseq/mi_vs_cell_line/adata_25c.h5ad')
scnsbm.inference.nested_model(adata25,collect_marginals=False)
adata25.write('mi_dropseq/mi_vs_cell_line/adata_25d.h5ad')
scnsbm.inference.nested_model(adata25,collect_marginals=False)
adata25.write('mi_dropseq/mi_vs_cell_line/adata_25e.h5ad')

In [ ]:
adata25a=sc.read('mi_dropseq/mi_vs_cell_line/adata_25a.h5ad')
adata25b=sc.read('mi_dropseq/mi_vs_cell_line/adata_25b.h5ad')
adata25c=sc.read('mi_dropseq/mi_vs_cell_line/adata_25c.h5ad')
adata25d=sc.read('mi_dropseq/mi_vs_cell_line/adata_25d.h5ad')
adata25e=sc.read('mi_dropseq/mi_vs_cell_line/adata_25e.h5ad')

### MI25

In [ ]:
nsbm_levels = [x for x in adata25a.obs_keys() if x.startswith("nsbm")]
foobars=['foobar25a','foobar25b','foobar25c','foobar25d','foobar25e']
foobar25a= [sklearn.metrics.adjusted_mutual_info_score(adata25a.obs.loc[:, NL], adata25a.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar25b= [sklearn.metrics.adjusted_mutual_info_score(adata25b.obs.loc[:, NL], adata25b.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar25c= [sklearn.metrics.adjusted_mutual_info_score(adata25c.obs.loc[:, NL], adata25c.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar25d= [sklearn.metrics.adjusted_mutual_info_score(adata25d.obs.loc[:, NL], adata25d.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar25e= [sklearn.metrics.adjusted_mutual_info_score(adata25e.obs.loc[:, NL], adata25e.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
MI_25=pd.DataFrame(None,columns=foobars,index=nsbm_levels)
MI_25.loc[:,'foobar25a']=foobar25a
MI_25.loc[:,'foobar25b']=foobar25b
MI_25.loc[:,'foobar25c']=foobar25c
MI_25.loc[:,'foobar25d']=foobar25d
MI_25.loc[:,'foobar25e']=foobar25e
mean_25=[mean(MI_25.iloc[x,:]) for x in range(len(nsbm_levels))]
sd_25=[statistics.stdev(MI_25.iloc[x,:]) for x in range(len(nsbm_levels))]
MI_25['mean_25']=mean_25
MI_25['standard_deviation_25']=sd_25
pickle.dump(MI_25, open('mi_dropseq/mi_vs_cell_line/mi_25.pkl','wb'),2)
print(MI_25)
MI_25.iloc[:,0:4].T.boxplot()

# ADATA30

In [ ]:
scnsbm.inference.nested_model(adata30)
adata30.write('mi_dropseq/mi_vs_cell_line/adata_30a.h5ad')
scnsbm.inference.nested_model(adata30,collect_marginals=False)
adata30.write('mi_dropseq/mi_vs_cell_line/adata_30b.h5ad')
scnsbm.inference.nested_model(adata30,collect_marginals=False)
adata30.write('mi_dropseq/mi_vs_cell_line/adata_30c.h5ad')
scnsbm.inference.nested_model(adata30,collect_marginals=False)
adata30.write('mi_dropseq/mi_vs_cell_line/adata_30d.h5ad')
scnsbm.inference.nested_model(adata30,collect_marginals=False)
adata30.write('mi_dropseq/mi_vs_cell_line/adata_30e.h5ad')

In [ ]:
adata30a=sc.read('mi_dropseq/mi_vs_cell_line/adata_30a.h5ad')
adata30b=sc.read('mi_dropseq/mi_vs_cell_line/adata_30b.h5ad')
adata30c=sc.read('mi_dropseq/mi_vs_cell_line/adata_30c.h5ad')
adata30d=sc.read('mi_dropseq/mi_vs_cell_line/adata_30d.h5ad')
adata30e=sc.read('mi_dropseq/mi_vs_cell_line/adata_30e.h5ad')

### MI30

In [ ]:
nsbm_levels = [x for x in adata30a.obs_keys() if x.startswith("nsbm")]
foobars=['foobar30a','foobar30b','foobar30c','foobar30d','foobar30e']
foobar30a= [sklearn.metrics.adjusted_mutual_info_score(adata30a.obs.loc[:, NL], adata30a.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar30b= [sklearn.metrics.adjusted_mutual_info_score(adata30b.obs.loc[:, NL], adata30b.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar30c= [sklearn.metrics.adjusted_mutual_info_score(adata30c.obs.loc[:, NL], adata30c.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar30d= [sklearn.metrics.adjusted_mutual_info_score(adata30d.obs.loc[:, NL], adata30d.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar30e= [sklearn.metrics.adjusted_mutual_info_score(adata30e.obs.loc[:, NL], adata30e.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
MI_30=pd.DataFrame(None,columns=foobars,index=nsbm_levels)
MI_30.loc[:,'foobar30a']=foobar30a
MI_30.loc[:,'foobar30b']=foobar30b
MI_30.loc[:,'foobar30c']=foobar30c
MI_30.loc[:,'foobar30d']=foobar30d
MI_30.loc[:,'foobar30e']=foobar30e
mean_30=[mean(MI_30.iloc[x,:]) for x in range(len(nsbm_levels))]
sd_30=[statistics.stdev(MI_30.iloc[x,:]) for x in range(len(nsbm_levels))]
MI_30['mean_30']=mean_30
MI_30['standard_deviation_30']=sd_30
pickle.dump(MI_30, open('mi_dropseq/mi_vs_cell_line/mi_30.pkl','wb'),2)
print(MI_30)
MI_30.iloc[:,0:4].T.boxplot()

# ADATA35

In [ ]:
scnsbm.inference.nested_model(adata35)
adata35.write('mi_dropseq/mi_vs_cell_line/adata_35a.h5ad')
scnsbm.inference.nested_model(adata35,collect_marginals=False)
adata35.write('mi_dropseq/mi_vs_cell_line/adata_35b.h5ad')
scnsbm.inference.nested_model(adata35,collect_marginals=False)
adata35.write('mi_dropseq/mi_vs_cell_line/adata_35c.h5ad')
scnsbm.inference.nested_model(adata35,collect_marginals=False)
adata35.write('mi_dropseq/mi_vs_cell_line/adata_35d.h5ad')
scnsbm.inference.nested_model(adata35,collect_marginals=False)
adata35.write('mi_dropseq/mi_vs_cell_line/adata_35e.h5ad')

In [ ]:
adata35a=sc.read('mi_dropseq/mi_vs_cell_line/adata_35a.h5ad')
adata35b=sc.read('mi_dropseq/mi_vs_cell_line/adata_35b.h5ad')
adata35c=sc.read('mi_dropseq/mi_vs_cell_line/adata_35c.h5ad')
adata35d=sc.read('mi_dropseq/mi_vs_cell_line/adata_35d.h5ad')
adata35e=sc.read('mi_dropseq/mi_vs_cell_line/adata_35e.h5ad')

### MI35

In [ ]:
nsbm_levels = [x for x in adata35a.obs_keys() if x.startswith("nsbm")]
foobars=['foobar35a','foobar35b','foobar35c','foobar35d','foobar35e']
foobar35a= [sklearn.metrics.adjusted_mutual_info_score(adata35a.obs.loc[:, NL], adata35a.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar35b= [sklearn.metrics.adjusted_mutual_info_score(adata35b.obs.loc[:, NL], adata35b.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar35c= [sklearn.metrics.adjusted_mutual_info_score(adata35c.obs.loc[:, NL], adata35c.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar35d= [sklearn.metrics.adjusted_mutual_info_score(adata35d.obs.loc[:, NL], adata35d.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar35e= [sklearn.metrics.adjusted_mutual_info_score(adata35e.obs.loc[:, NL], adata35e.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
MI_35=pd.DataFrame(None,columns=foobars,index=nsbm_levels)
MI_35.loc[:,'foobar35a']=foobar35a
MI_35.loc[:,'foobar35b']=foobar35b
MI_35.loc[:,'foobar35c']=foobar35c
MI_35.loc[:,'foobar35d']=foobar35d
MI_35.loc[:,'foobar35e']=foobar35e
mean_35=[mean(MI_35.iloc[x,:]) for x in range(len(nsbm_levels))]
sd_35=[statistics.stdev(MI_35.iloc[x,:]) for x in range(len(nsbm_levels))]
MI_35['mean_35']=mean_35
MI_35['standard_deviation_35']=sd_35
pickle.dump(MI_35, open('mi_dropseq/mi_vs_cell_line/mi_35.pkl','wb'),2)
print(MI_35)
MI_35.iloc[:,0:4].T.boxplot()

# ADATA40

In [ ]:
scnsbm.inference.nested_model(adata40)
adata40.write('mi_dropseq/mi_vs_cell_line/adata_40a.h5ad')
scnsbm.inference.nested_model(adata40,collect_marginals=False)
adata40.write('mi_dropseq/mi_vs_cell_line/adata_40b.h5ad')
scnsbm.inference.nested_model(adata40,collect_marginals=False)
adata40.write('mi_dropseq/mi_vs_cell_line/adata_40c.h5ad')
scnsbm.inference.nested_model(adata40,collect_marginals=False)
adata40.write('mi_dropseq/mi_vs_cell_line/adata_40d.h5ad')
scnsbm.inference.nested_model(adata40,collect_marginals=False)
adata40.write('mi_dropseq/mi_vs_cell_line/adata_40e.h5ad')

In [ ]:
adata40a=sc.read('mi_dropseq/mi_vs_cell_line/adata_40a.h5ad')
adata40b=sc.read('mi_dropseq/mi_vs_cell_line/adata_40b.h5ad')
adata40c=sc.read('mi_dropseq/mi_vs_cell_line/adata_40c.h5ad')
adata40d=sc.read('mi_dropseq/mi_vs_cell_line/adata_40d.h5ad')
adata40e=sc.read('mi_dropseq/mi_vs_cell_line/adata_40e.h5ad')

### MI40

In [ ]:
nsbm_levels = [x for x in adata40a.obs_keys() if x.startswith("nsbm")]
foobars=['foobar40a','foobar40b','foobar40c','foobar40d','foobar40e']
foobar40a= [sklearn.metrics.adjusted_mutual_info_score(adata40a.obs.loc[:, NL], adata40a.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar40b= [sklearn.metrics.adjusted_mutual_info_score(adata40b.obs.loc[:, NL], adata40b.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar40c= [sklearn.metrics.adjusted_mutual_info_score(adata40c.obs.loc[:, NL], adata40c.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar40d= [sklearn.metrics.adjusted_mutual_info_score(adata40d.obs.loc[:, NL], adata40d.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar40e= [sklearn.metrics.adjusted_mutual_info_score(adata40e.obs.loc[:, NL], adata40e.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
MI_40=pd.DataFrame(None,columns=foobars,index=nsbm_levels)
MI_40.loc[:,'foobar40a']=foobar40a
MI_40.loc[:,'foobar40b']=foobar40b
MI_40.loc[:,'foobar40c']=foobar40c
MI_40.loc[:,'foobar40d']=foobar40d
MI_40.loc[:,'foobar40e']=foobar40e
mean_40=[mean(MI_40.iloc[x,:]) for x in range(len(nsbm_levels))]
sd_40=[statistics.stdev(MI_40.iloc[x,:]) for x in range(len(nsbm_levels))]
MI_40['mean_40']=mean_40
MI_40['standard_deviation_40']=sd_40
pickle.dump(MI_40, open('mi_dropseq/mi_vs_cell_line/mi_40.pkl','wb'),2)
print(MI_40)
MI_40.iloc[:,0:4].T.boxplot()

# ADATA45

In [ ]:
scnsbm.inference.nested_model(adata45)
adata45.write('mi_dropseq/mi_vs_cell_line/adata_45a.h5ad')
scnsbm.inference.nested_model(adata45,collect_marginals=False)
adata45.write('mi_dropseq/mi_vs_cell_line/adata_45b.h5ad')
scnsbm.inference.nested_model(adata45,collect_marginals=False)
adata45.write('mi_dropseq/mi_vs_cell_line/adata_45c.h5ad')
scnsbm.inference.nested_model(adata45,collect_marginals=False)
adata45.write('mi_dropseq/mi_vs_cell_line/adata_45d.h5ad')
scnsbm.inference.nested_model(adata45,collect_marginals=False)
adata45.write('mi_dropseq/mi_vs_cell_line/adata_45e.h5ad')

In [ ]:
adata45a=sc.read('mi_dropseq/mi_vs_cell_line/adata_45a.h5ad')
adata45b=sc.read('mi_dropseq/mi_vs_cell_line/adata_45b.h5ad')
adata45c=sc.read('mi_dropseq/mi_vs_cell_line/adata_45c.h5ad')
adata45d=sc.read('mi_dropseq/mi_vs_cell_line/adata_45d.h5ad')
adata45e=sc.read('mi_dropseq/mi_vs_cell_line/adata_45e.h5ad')

### MI45

In [ ]:
nsbm_levels = [x for x in adata45a.obs_keys() if x.startswith("nsbm")]
foobars=['foobar45a','foobar45b','foobar45c','foobar45d','foobar45e']
foobar45a= [sklearn.metrics.adjusted_mutual_info_score(adata45a.obs.loc[:, NL], adata45a.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar45b= [sklearn.metrics.adjusted_mutual_info_score(adata45b.obs.loc[:, NL], adata45b.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar45c= [sklearn.metrics.adjusted_mutual_info_score(adata45c.obs.loc[:, NL], adata45c.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar45d= [sklearn.metrics.adjusted_mutual_info_score(adata45d.obs.loc[:, NL], adata45d.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar45e= [sklearn.metrics.adjusted_mutual_info_score(adata45e.obs.loc[:, NL], adata45e.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
MI_45=pd.DataFrame(None,columns=foobars,index=nsbm_levels)
MI_45.loc[:,'foobar45a']=foobar45a
MI_45.loc[:,'foobar45b']=foobar45b
MI_45.loc[:,'foobar45c']=foobar45c
MI_45.loc[:,'foobar45d']=foobar45d
MI_45.loc[:,'foobar45e']=foobar45e
mean_45=[mean(MI_45.iloc[x,:]) for x in range(len(nsbm_levels))]
sd_45=[statistics.stdev(MI_45.iloc[x,:]) for x in range(len(nsbm_levels))]
MI_45['mean_45']=mean_45
MI_45['standard_deviation_45']=sd_45
pickle.dump(MI_45, open('mi_dropseq/mi_vs_cell_line/mi_45.pkl','wb'),2)
print(MI_45)
MI_45.iloc[:,0:4].T.boxplot()

# ADATA50

In [ ]:
scnsbm.inference.nested_model(adata50)
adata50.write('mi_dropseq/mi_vs_cell_line/adata_50a.h5ad')
scnsbm.inference.nested_model(adata50,collect_marginals=False)
adata50.write('mi_dropseq/mi_vs_cell_line/adata_50b.h5ad')
scnsbm.inference.nested_model(adata50,collect_marginals=False)
adata50.write('mi_dropseq/mi_vs_cell_line/adata_50c.h5ad')
scnsbm.inference.nested_model(adata50,collect_marginals=False)
adata50.write('mi_dropseq/mi_vs_cell_line/adata_50d.h5ad')
scnsbm.inference.nested_model(adata50,collect_marginals=False)
adata50.write('mi_dropseq/mi_vs_cell_line/adata_50e.h5ad')

In [ ]:
adata50a=sc.read('mi_dropseq/mi_vs_cell_line/adata_50a.h5ad')
adata50b=sc.read('mi_dropseq/mi_vs_cell_line/adata_50b.h5ad')
adata50c=sc.read('mi_dropseq/mi_vs_cell_line/adata_50c.h5ad')
adata50d=sc.read('mi_dropseq/mi_vs_cell_line/adata_50d.h5ad')
adata50e=sc.read('mi_dropseq/mi_vs_cell_line/adata_50e.h5ad')

### MI50

In [ ]:
nsbm_levels = [x for x in adata50a.obs_keys() if x.startswith("nsbm")]
foobars=['foobar50a','foobar50b','foobar50c','foobar50d','foobar50e']
foobar50a= [sklearn.metrics.adjusted_mutual_info_score(adata50a.obs.loc[:, NL], adata50a.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar50b= [sklearn.metrics.adjusted_mutual_info_score(adata50b.obs.loc[:, NL], adata50b.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar50c= [sklearn.metrics.adjusted_mutual_info_score(adata50c.obs.loc[:, NL], adata50c.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar50d= [sklearn.metrics.adjusted_mutual_info_score(adata50d.obs.loc[:, NL], adata50d.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
foobar50e= [sklearn.metrics.adjusted_mutual_info_score(adata50e.obs.loc[:, NL], adata50e.obs.loc[:, 'cell_line'])
                                            for NL in nsbm_levels]
MI_50=pd.DataFrame(None,columns=foobars,index=nsbm_levels)
MI_50.loc[:,'foobar50a']=foobar50a
MI_50.loc[:,'foobar50b']=foobar50b
MI_50.loc[:,'foobar50c']=foobar50c
MI_50.loc[:,'foobar50d']=foobar50d
MI_50.loc[:,'foobar50e']=foobar50e
mean_50=[mean(MI_50.iloc[x,:]) for x in range(len(nsbm_levels))]
sd_50=[statistics.stdev(MI_50.iloc[x,:]) for x in range(len(nsbm_levels))]
MI_50['mean_50']=mean_50
MI_50['standard_deviation_50']=sd_50
pickle.dump(MI_50, open('mi_dropseq/mi_vs_cell_line/mi_50.pkl','wb'),2)
print(MI_50)
MI_50.iloc[:,0:4].T.boxplot()